In [1]:
import mysql.connector
import pandas as pd

# 데이터베이스 연결 설정
config = {
  'user': 'root',
  'password': '00000000',
  'host': 'localhost',
  'database': 'ZeroBase',
  'raise_on_warnings': True
}

# 연결 생성
connection = mysql.connector.connect(**config)

# 커서 생성
cursor = connection.cursor()

# 여기서 쿼리를 실행하거나 다른 작업을 수행할 수 있습니다.
cursor.execute('''select count(*)
                  from (
                        select customer_unique_id, count(customer_unique_id) as customer_count
                        from olist_customers_dataset
                        group by customer_unique_id
                        having count(customer_unique_id) > 1
                        order by count(customer_unique_id)
                  ) a
''')
rows = cursor.fetchall()
for row in rows:
    print(row)

# 커서 및 연결 닫기
cursor.close()
# connection.close()


(2997,)


True

* 고객 중 2번 이상 구매한 고객은 2,997명이다.

In [2]:
# 커서 생성
cursor = connection.cursor()

# 여기서 쿼리를 실행하거나 다른 작업을 수행할 수 있습니다.
cursor.execute('''select count(*)
                  from olist_customers_dataset a
                  where a.customer_id not in (
                    select b.customer_id
                    from olist_orders_dataset b
                  )
''')
rows = cursor.fetchall()
for row in rows:
    print(row)

# 커서 및 연결 닫기
cursor.close()

(0,)


True

* 고객 테이블 고객id와 주문 테이블 고객id가 1:1 대응이다. 즉, 미사용 고객은 없다.

In [3]:
# 커서 생성
cursor = connection.cursor()

# 여기서 쿼리를 실행하거나 다른 작업을 수행할 수 있습니다.
cursor.execute('''select
                    b.customer_unique_id,
                    a.customer_id,
                    a.order_id,
                    a.order_status,
                    a.order_purchase_timestamp,
                    a.order_approved_at,
                    a.order_delivered_carrier_date,
                    a.order_delivered_customer_date,
                    a.order_estimated_delivery_date
                  from olist_orders_dataset a
                  left join olist_customers_dataset b
                  on a.customer_id = b.customer_id
                  group by b.customer_unique_id, a.customer_id, a.order_id, a.order_status, a.order_purchase_timestamp, a.order_approved_at, a.order_delivered_carrier_date, a.order_delivered_customer_date, a.order_estimated_delivery_date
                  order by b.customer_unique_id
''')

# 컬럼명 가져오기
columns = [desc[0] for desc in cursor.description]

# 결과를 데이터프레임으로 저장
df_one_on_one = pd.DataFrame(cursor.fetchall(), columns=columns)

# 커서 및 연결 닫기
cursor.close()

df_one_on_one

,customer_unique_id,customer_id,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0000366f3b9a7992bf8c76cfdf3221e2,fadbb3709178fc513abc1b2670aa1ad2,e22acc9c116caa3f2b7121bbb380d08e,delivered,2018-05-10 10:56:27,2018-05-10 11:11:18,2018-05-12 08:18:00,2018-05-16 20:48:37,2018-05-21 00:00:00
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4cb282e167ae9234755102258dd52ee8,3594e05a005ac4d06a72673270ef9ec9,delivered,2018-05-07 11:11:27,2018-05-07 18:25:44,2018-05-09 12:18:00,2018-05-10 18:02:42,2018-05-15 00:00:00
2,0000f46a3911fa3c0805444483337064,9b3932a6253894a02c1df9d19004239f,b33ec3b699337181488304f362a6b734,delivered,2017-03-10 21:05:03,2017-03-10 21:05:03,2017-03-13 12:58:30,2017-04-05 14:38:47,2017-04-07 00:00:00
3,0000f6ccb0745a6a4b88665a16c9f078,914991f0c02ef0843c0e7010c819d642,41272756ecddd9a9ed0180413cc22fb6,delivered,2017-10-12 20:29:41,2017-10-12 20:49:17,2017-10-13 20:08:19,2017-11-01 21:23:05,2017-11-13 00:00:00
4,0004aac84e0df4da2b147fca70cf8255,47227568b10f5f58a524a75507e6992c,d957021f1127559cd947b62533f484f7,delivered,2017-11-14 19:45:42,2017-11-14 20:06:52,2017-11-16 19:52:10,2017-11-27 23:08:56,2017-12-05 00:00:00
...,...,...,...,...,...,...,...,...,...
99436,fffcf5a5ff07b0908bd4e2dbc735a684,74be082247cd677a147d83ee670e9d53,725cf8e9c24e679a8a5a32cb92c9ce1e,delivered,2017-06-08 21:00:36,2017-06-08 21:15:16,2017-06-23 13:03:27,2017-07-06 09:22:00,2017-08-02 00:00:00
99437,fffea47cd6d3cc0a88bd621562a9d061,0ecf8e0a08148af0bf313184f167670a,c71b9252fd7b3b263aaa4cb09319a323,delivered,2017-12-10 20:07:56,2017-12-10 20:16:20,2017-12-21 16:27:01,2018-01-09 22:28:20,2018-01-12 00:00:00
99438,ffff371b4d645b6ecea244b27531430a,27f584b0f1dc4e610065c240f68b6be0,fdc45e6c7555e6cb3cc0daca2557dbe1,delivered,2017-02-07 15:49:16,2017-02-07 16:02:44,2017-02-08 10:04:37,2017-02-22 12:45:04,2017-03-24 00:00:00
99439,ffff5962728ec6157033ef9805bacc48,832a3b0254347d409512ae92eaf154a6,94d3ee0bc2a0af9d4fa47a4d63616e8d,delivered,2018-05-02 15:17:41,2018-05-02 15:58:47,2018-05-03 16:45:00,2018-05-14 11:54:26,2018-05-28 00:00:00


* 고객이 주문 할 때 customer_id가 생기고, 이는 order_id와 결합된다.
* 즉, 한 건의 주문(order_id)가 생성되면, customer_id가 하나 생성된다.

In [4]:
# 커서 생성
cursor = connection.cursor()

# 여기서 쿼리를 실행하거나 다른 작업을 수행할 수 있습니다.
cursor.execute('''select distinct customer_unique_id
                  from olist_customers_dataset
                  where customer_unique_id not in (
                    select distinct e.customer_unique_id
                    from olist_customers_dataset e
                    left join olist_orders_dataset f
                    on e.customer_id = f.customer_id
                    where DATE_FORMAT(f.order_purchase_timestamp, '%Y-%m-%d') >= '2018-01-01'
                  )
''')

# 컬럼명 가져오기
columns = [desc[0] for desc in cursor.description]

# 휴면고객 고유id 명단
# 고유id로 뽑은 이유: 한 사람이 두 건 이상 주문했을 시 customer_id는 새로 생성되기 때문에 고유id로 개인이 몇 번 주문했는지 알 수 있다.
df_sleep_customer_unique_id = pd.DataFrame(cursor.fetchall(), columns=columns)

# 커서 및 연결 닫기
cursor.close()

df_sleep_customer_unique_id

,customer_unique_id
0,861eff4711a542e4b93843c6dd7febb0
1,4c93744516667ad3b8f1fb645a3116a4
2,57b2a98a409812fe9618067b6b8ebe4f
3,2a7745e1ed516b289ed9b29c7d0539a5
4,918dc87cd72cd9f6ed4bd442ed785235
...,...
43342,277490f0d435b602fe4475d4b89e9181
43343,e7f8760e2bbd2f1986bebd99596c088e
43344,4b5820135d360a45552b5163835b1d89
43345,73c2643a0a458b49f58cea58833b192e


* 휴면고객은 총 43,347명이다.

In [6]:
# 커서 생성
cursor = connection.cursor()

# 여기서 쿼리를 실행하거나 다른 작업을 수행할 수 있습니다.
cursor.execute('''select
                    b.customer_unique_id,
                    a.customer_id,
                    a.order_id,
                    a.order_status,
                    a.order_purchase_timestamp,
                    a.order_approved_at,
                    a.order_delivered_carrier_date,
                    a.order_delivered_customer_date,
                    a.order_estimated_delivery_date
                  from olist_orders_dataset a
                  left join olist_customers_dataset b
                  on a.customer_id = b.customer_id
                  where b.customer_unique_id in (
                    select distinct customer_unique_id
                    from olist_customers_dataset
                    where customer_unique_id not in (
                        select distinct e.customer_unique_id
                        from olist_customers_dataset e
                        left join olist_orders_dataset f
                        on e.customer_id = f.customer_id
                        where DATE_FORMAT(f.order_purchase_timestamp, '%Y-%m-%d') >= '2018-01-01'
                    )
                  )
                  group by 
                    b.customer_unique_id,
                    a.customer_id,
                    a.order_id,
                    a.order_status,
                    a.order_purchase_timestamp,
                    a.order_approved_at,
                    a.order_delivered_carrier_date,
                    a.order_delivered_customer_date,
                    a.order_estimated_delivery_date
                  order by b.customer_unique_id
''')

# 컬럼명 가져오기
columns = [desc[0] for desc in cursor.description]

# 휴면고객 데이터셋
df_sleep_customer_dataset = pd.DataFrame(cursor.fetchall(), columns=columns)

# 커서 및 연결 닫기
cursor.close()

df_sleep_customer_dataset

,customer_unique_id,customer_id,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0000f46a3911fa3c0805444483337064,9b3932a6253894a02c1df9d19004239f,b33ec3b699337181488304f362a6b734,delivered,2017-03-10 21:05:03,2017-03-10 21:05:03,2017-03-13 12:58:30,2017-04-05 14:38:47,2017-04-07 00:00:00
1,0000f6ccb0745a6a4b88665a16c9f078,914991f0c02ef0843c0e7010c819d642,41272756ecddd9a9ed0180413cc22fb6,delivered,2017-10-12 20:29:41,2017-10-12 20:49:17,2017-10-13 20:08:19,2017-11-01 21:23:05,2017-11-13 00:00:00
2,0004aac84e0df4da2b147fca70cf8255,47227568b10f5f58a524a75507e6992c,d957021f1127559cd947b62533f484f7,delivered,2017-11-14 19:45:42,2017-11-14 20:06:52,2017-11-16 19:52:10,2017-11-27 23:08:56,2017-12-05 00:00:00
3,0005e1862207bf6ccc02e4228effd9a0,3b37fb626fdf46cd99d37ec62afa88ff,ae76bef74b97bcb0b3e355e60d9a6f9c,delivered,2017-03-04 23:32:12,2017-03-04 23:43:26,2017-03-06 05:14:07,2017-03-09 08:33:08,2017-04-06 00:00:00
4,0006fdc98a402fceb4eb0ee528f6a8d4,2f29573c8cac5a7be11c5b649078f944,6681163e3dab91c549952b2845b20281,delivered,2017-07-18 09:23:10,2017-07-18 09:30:22,2017-07-21 16:13:08,2017-08-03 18:42:49,2017-08-15 00:00:00
...,...,...,...,...,...,...,...,...,...
44634,fffbf87b7a1a6fa8b03f081c5f51a201,ed421f2bf6c0ea955e4ac8a5fe646b94,64397307c6954ae1ad2ad8e791ad8a31,delivered,2017-12-27 22:36:41,2017-12-27 22:47:44,2018-01-05 00:44:26,2018-01-15 19:39:13,2018-01-29 00:00:00
44635,fffcf5a5ff07b0908bd4e2dbc735a684,74be082247cd677a147d83ee670e9d53,725cf8e9c24e679a8a5a32cb92c9ce1e,delivered,2017-06-08 21:00:36,2017-06-08 21:15:16,2017-06-23 13:03:27,2017-07-06 09:22:00,2017-08-02 00:00:00
44636,fffea47cd6d3cc0a88bd621562a9d061,0ecf8e0a08148af0bf313184f167670a,c71b9252fd7b3b263aaa4cb09319a323,delivered,2017-12-10 20:07:56,2017-12-10 20:16:20,2017-12-21 16:27:01,2018-01-09 22:28:20,2018-01-12 00:00:00
44637,ffff371b4d645b6ecea244b27531430a,27f584b0f1dc4e610065c240f68b6be0,fdc45e6c7555e6cb3cc0daca2557dbe1,delivered,2017-02-07 15:49:16,2017-02-07 16:02:44,2017-02-08 10:04:37,2017-02-22 12:45:04,2017-03-24 00:00:00
